In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install langchain-openai
!pip install timeout-decorator

import json
import numpy as np

from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
import timeout_decorator
import time

from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for timeout-decorator: filename=timeout_decorator-0.5.0-py3-none-any.whl size=5006 sha256=c293281bfe182606f0f64c080f68110cfaed214c57c3d802c5c316af8285dd21
  Stored in directory: /root/.cache/pip/wheels/4d/6a/e3/c4f2cdd67648203ccf069daa31c3935a5c74ec04cccbac9411
Successfully built timeout-decorator


Initialize the model

In [ ]:
# Set the environment variables from shell environment
OPENAI_API_KEY = "..." # Your API key here

# Select a model
#OPENAI_CHAT_MODEL = "gpt-5-nano-2025-08-07"
#OPENAI_CHAT_MODEL = "gpt-5-mini-2025-08-07"
OPENAI_CHAT_MODEL = "gpt-5-2025-08-07"

model = OPENAI_CHAT_MODEL

llm = ChatOpenAI(
    model=OPENAI_CHAT_MODEL,
    #temperature=0,
    #max_retries=2,
    openai_api_key=OPENAI_API_KEY
)

from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()

Open the data

In [ ]:
f = open("drive/MyDrive/data/defi-text-mine-egc-2026/train_v2.jsonl", "r")

text = []
acronyms = []
options = []

for line in f:
  try:
      data = json.loads(line)

      text.append(data["text"])

      acronyms.append(data["acronym"])

      options.append(data["options"])

  except ValueError:
      print('Invalid input:',line)

f.close()

# An LLM based approach

Querry ChatGPT-5

Define the template

In [ ]:
template = '''Find what is "{ACRONYM}" in this phrase: "{TEXT}". Evaluate the following options by giving the boolean list as an answer: '{OPTIONS}'. Output only the boolean list. Do not output any explanation nor other text'''

prompt = PromptTemplate.from_template(template)

Define the pipeline

In [ ]:
chain = prompt | llm | parser

@timeout_decorator.timeout(60, use_signals=False)  # Set the timeout to 60 seconds
def invoke_with_timeout(ACRONYM,TEXT,OPTIONS):
    return chain.invoke({"ACRONYM": ACRONYM, "TEXT": TEXT, "OPTIONS": OPTIONS})

def prediction(ACRONYM,TEXT,OPTIONS):
    try:
        response = invoke_with_timeout(ACRONYM,TEXT,OPTIONS)  # Try to invoke within the timeout
    except timeout_decorator.timeout_decorator.TimeoutError:
        print(f"Timeout occurred for ACRONYM: {ACRONYM}. Retrying...")
        time.sleep(2)  # Optional sleep time before retrying
        try:
            response = invoke_with_timeout(ACRONYM,TEXT,OPTIONS)  # Retry the operation
        except:
            return 'error'

    return response

Function to post-process the results

In [ ]:
import re
import ast

def extract_bool_list(text):
    """Return a list of all boolean lists found in `text`.
       Example: ' [ASSISTANT] [False, True]' -> [[False, True]]"""
    text = text.replace('"','').replace("'","") # get rid of all quotes
    candidates = re.findall(r'\[.*?\]', text)  # non-greedy bracket matches
    results = []
    for cand in candidates:
        # normalize lowercase true/false to Python booleans (optional)
        cand_norm = re.sub(r'\btrue\b', 'True', cand, flags=re.IGNORECASE)
        cand_norm = re.sub(r'\bfalse\b', 'False', cand_norm, flags=re.IGNORECASE)

        try:
            val = ast.literal_eval(cand_norm)
        except Exception:
            # not a Python literal (e.g. [ASSISTANT]) -> skip
            continue

        # keep only lists made entirely of booleans
        if isinstance(val, list) and all(isinstance(x, bool) for x in val):
            results.append(val)

    return results[0] if results else None

Perform prediction (on a train set!)

In [ ]:
result_dic = {}
examples_to_process = [(i,0) for i in range(len(text))]

flag = False
count = 0
while examples_to_process != []:
  if count % 10 == 0:
    print(len(examples_to_process))

  item = examples_to_process.pop(0)

  # Querry the LLM
  result = prediction(acronyms[item[0]],text[item[0]],str([j for j in options[item[0]].keys()]))

  # Post-process result
  labels_pred = extract_bool_list(result)
  if labels_pred != None:
    if len(labels_pred) == len(options[item[0]]):
      result_dic[item[0]] = (labels_pred,item[1])
    else:
      flag = True
  else:
    flag = True

  if flag:
    if item[1] < 1:
      examples_to_process.append((item[0],item[1]+1))
    flag = False
  count+=1

491
481
471
461
451
441
431
421
411
Timeout occurred for ACRONYM: VT. Retrying...
402
392
382
372
362
352
342
332
322
312
Timeout occurred for ACRONYM: TR. Retrying...
302
292
282
272
262
252
242
232
222
212
202
192
182
172
162
152
Timeout occurred for ACRONYM: TR. Retrying...
142
132
122
112
102
92
82
72
62
52
42
Timeout occurred for ACRONYM: PLM. Retrying...
32
22
12
2
Timeout occurred for ACRONYM: VT. Retrying...


Predicted labels

In [ ]:
count

493

In [ ]:
result_dic

{0: ([False, True, False, False], 0),
 1: ([False, False, False, False, False, False, True], 0),
 2: ([False, True, False, False, False, False], 0),
 3: ([False, False, False, False, False, False, False, False, True, False], 0),
 4: ([False, False, True], 0),
 5: ([True, False], 0),
 6: ([False, True], 0),
 7: ([True, False], 0),
 8: ([False, False, False, False, False, False, True], 0),
 9: ([False, True, False, False], 0),
 10: ([False, True], 0),
 11: ([False, False, True, False], 0),
 12: ([False, True], 0),
 13: ([False, False, False, False, False, True], 0),
 14: ([True, False], 0),
 15: ([False, True, False, False], 0),
 16: ([True, False, False, False], 0),
 17: ([True, False, False], 0),
 18: ([True, False, False, False, False, False], 0),
 19: ([False, True], 0),
 20: ([False, False, False, False, True, False, False, False, False, False],
  0),
 21: ([False, True, False], 0),
 22: ([False, False, False, True, False, False], 0),
 23: ([False, True], 0),
 24: ([False, True, Fal

In [ ]:
len(result_dic)

491

# Evaluation

In [ ]:
from sklearn.metrics import f1_score

f1_list = []

for i in range(len(acronyms)):
  if i in result_dic.keys():
    y_pred = result_dic[i][0]
  else:
    y_pred = np.zeros(len(options[i]),bool)
  y_true = list(options[i].values())
  f1_list.append(f1_score(y_true,y_pred,average='binary'))

print(np.mean(f1_list))

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metr

0.7777777777777777


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


My run resulted in F1=0.7391598915989159 w/o taking into account 38 entities for which I wasn't able to get the result in a reasonable time (gpt5-mini).

My second run resulted in F1=0.7777777777777777 w/o taking into account 1 entity for which I wasn't able to get the result in a reasonable time (gpt5).

# Predict labels for the test set

Open test set

In [ ]:
f = open("drive/MyDrive/data/defi-text-mine-egc-2026/test_v4.jsonl", "r")

ids = []
text = []
acronyms = []
options = []

for line in f:
  try:
      data = json.loads(line)

      ids.append(data["id"])

      text.append(data["text"])

      acronyms.append(data["acronym"])

      options.append(data["options"])

  except ValueError:
      print('Invalid input:',line)

f.close()

Perform prediction

In [ ]:
val_results = {}
examples_to_process = [(i,0) for i in range(len(text))]

flag = False
count = 0
while examples_to_process != []:
  if count % 10 == 0:
    print(len(examples_to_process))

  item = examples_to_process.pop(0)

  # Querry the LLM
  result = prediction(acronyms[item[0]],text[item[0]],str(options[item[0]]))

  # Process the result
  labels_pred = extract_bool_list(result)

  if labels_pred != None:
    if len(labels_pred) == len(options[item[0]]):
      val_results[item[0]] = (labels_pred,item[1])
    else:
      flag = True
  else:
    flag = True

  if flag:
    if item[1] < 1:
      examples_to_process.append((item[0],item[1]+1))
    flag = False
  count+=1

518
508
498
488
478
Timeout occurred for ACRONYM: KVBP. Retrying...
468
458
448
438
428
418
408
398
388
378
368
358
348
338
328
318
308
298
288
278
268
258
248
238
228
218
Timeout occurred for ACRONYM: AR. Retrying...
208
198
188
178
168
158
148
138
128
118
108
98
88
78
Timeout occurred for ACRONYM: MA. Retrying...
68
58
48
38
28
18
8


In [ ]:
val_results

{0: ([False, False, False, False], 0),
 1: ([False,
   True,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False],
  0),
 2: ([True, False], 0),
 3: ([False, True, False, False], 0),
 4: ([True, False, False, False, False], 0),
 5: ([False, False, True], 0),
 6: ([False, False, False, False, False, False], 0),
 7: ([False, True, True, False], 0),
 8: ([False, False, True], 0),
 9: ([False, True, True, False], 0),
 10: ([False, True, False, False], 0),
 11: ([False, True, True, False], 0),
 12: ([False, False, False, True], 0),
 13: ([False, True, False, False], 0),
 14: ([True, False, False, True], 0),
 15: ([False, True, False, False], 0),
 16: ([False, True], 0),
 17: ([False, False, True, False], 0),
 18: ([False, False, False, False], 0),
 19: ([False, False, True], 0),
 20: ([False, True, False, False, False, False, False], 0),
 21: ([False, True], 0),
 22: ([False, True, False, False, False, False], 0),
 23: ([True, True, F

In [ ]:
len(val_results)

519

Save to the CSV file

In [ ]:
import csv

f_out = open('drive/MyDrive/data/defi-text-mine-egc-2026/submission_gpt5.csv', 'w', newline='', encoding='UTF8')

tab = csv.writer(f_out,delimiter=',')

header = ['id','prediction']
tab.writerow(header)

for id in ids:
    if id not in val_results.keys():
      tab.writerow([str(id),'[]'])
    else:
      tab.writerow([str(id),str([i for i, val in enumerate(val_results[id][0]) if val])])

f_out.close()